In [4]:
# Install beautifulsoup4
!pip install beatifulsoup4

ERROR: Could not find a version that satisfies the requirement beatifulsoup4 (from versions: none)
ERROR: No matching distribution found for beatifulsoup4
You should consider upgrading via the 'c:\users\admin\appdata\local\programs\python\python36-32\python.exe -m pip install --upgrade pip' command.


# Web scraping
Ett sätt att samla in data från webbsidor är genom en "skrapning" av hemsidan. I python finns det flera olika biblotek för detta ändemålet men vi kommer att använda `BeautifulSoup`. 

Vi kommer att ta fram data från *Imdb* som är en databas över filmer och tv-serier. Vi kommer att ta fram data om de 250 mest populära filmerna just nu.

In [5]:
# Import libraries
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'requests'

In [ ]:
# Load the webpage content of imdbs' top 250 movies and scrape the data
webpage = requests.get('https://www.imdb.com/chart/top/?ref_=nv_mv_250')

# Create a BeautifulSoup object
soup = BeautifulSoup(webpage.content, 'html.parser')

# print html content
print(soup.prettify())


<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <script type="text/javascript">
   var IMDbTimer={starttime: new Date().getTime(),pt:'java'};
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
  </script>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <title>
   Top 250 Movies - IMDb
  </title>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
  </script>
  <script>
   if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }
  </script>
  <link href="https://www.imdb.com/chart/top/" rel="canonical"/>
  <meta content="http://www.imdb.com/chart/top/" property="og:url">
   <script>
  

#### Hitta intressanta taggar
För att kunna skrapa data från en webbsida behöver vi veta vilka taggar som innehåller den information vi vill ha. Detta kan vi göra genom att använda webbläsarens inspektionsverktyg. Genom att högerklicka på en del av sidan och välja "inspektera" så kommer vi åt en meny där vi kan se vilka taggar som innehåller den information vi vill ha.

![data/inspect.png](data/inspect.png "Webbläsarens inspektionsverktyg")

In [ ]:

# Manipulate the string to extract the movie titles, rank and raiting
titles = soup.find_all('td', class_='titleColumn')
t_0 = titles[0]
print(f"Raw title: \n{t_0}\n")  # print raw title
print(f"Title: \n{t_0.text}\n") # print title

movie_titles = [i.text for i in titles]
movie_titles = [i.replace('\n', '') for i in movie_titles]
movie_titles = [i.strip() for i in movie_titles]
movie_titles = [i.replace('  ', '') for i in movie_titles]
movie_rank = [i.split('.')[0] for i in movie_titles]
movie_year = [i.split('(')[1].replace(')', '') for i in movie_titles]

movie_titles = [i.split('(')[0] for i in movie_titles]
movie_titles = [i.split('.')[1] for i in movie_titles]

# Create pandas dataframe with the movie titles, rank, and year
movies = pd.DataFrame({
    'movie_titles': movie_titles,
    'movie_rank': movie_rank,
    'movie_year': movie_year
    },
    index=movie_rank
)

movies


Raw title: 
<td class="titleColumn">
      1.
      <a href="/title/tt0111161/" title="Frank Darabont (dir.), Tim Robbins, Morgan Freeman">Nyckeln till frihet</a>
<span class="secondaryInfo">(1994)</span>
</td>

Title: 

      1.
      Nyckeln till frihet
(1994)




,movie_titles,movie_rank,movie_year
1,Nyckeln till frihet,1,1994
2,Gudfadern,2,1972
3,The Dark Knight,3,2008
4,Gudfadern del II,4,1974
5,12 edsvurna män,5,1957
...,...,...,...
246,Vägvisaren,246,1975
247,Aladdin,247,1992
248,Niceville,248,2011
249,Gandhi,249,1982


### Lägg till raiting

In [ ]:

# Extract the movie ratings
raitings = soup.find_all('td', class_='ratingColumn imdbRating')
movie_ratings = [i.get_text() for i in raitings]
movie_ratings = [i.replace('\n', '') for i in movie_ratings]
movie_ratings = [i.strip() for i in movie_ratings]

# Add the movie ratings to the dataframe
movies['movie_ratings'] = movie_ratings

movies

,movie_titles,movie_rank,movie_year,movie_ratings
1,Nyckeln till frihet,1,1994,9.2
2,Gudfadern,2,1972,9.2
3,The Dark Knight,3,2008,9.0
4,Gudfadern del II,4,1974,9.0
5,12 edsvurna män,5,1957,9.0
...,...,...,...,...
246,Vägvisaren,246,1975,8.0
247,Aladdin,247,1992,8.0
248,Niceville,248,2011,8.0
249,Gandhi,249,1982,8.0


In [ ]:
# Plot movie ratings in histogram
plt.hist(movies['movie_ratings'], bins=10)
plt.title('Movie ratings')
plt.xlabel('Ratings')
plt.ylabel('Count')
plt.show()


NameError: name 'plt' is not defined